# Data Prepration

In [1]:
from prepross import *
import numpy as np
import os
import dataloader
from dataloader import Vocab
pd.options.display.max_rows = 100
from embedding import Doc2VecEmbedding
import pickle
import numpy as np
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mehrazin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mehrazin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
class Parse():
    def __init__(self):
        self.test_mode = True
        self.load_vocab = True
arg = Parse()

In [3]:
config = Config(arg)

In [4]:
config.exp_dir

'/home/mehrazin/Documents/DAV_project/Repository/Music-Recommendation/Dumped/12'

In [5]:
config.Dataset_dir

'/home/mehrazin/Documents/DAV_project/Repository/Music-Recommendation/Dataset/Test/1k-item/Datasets'

In [6]:
config.device

device(type='cpu')

In [7]:
# df = Data_handler(config, pass_col = ['lyrics'])
# df.data = df.get_masked_data(users = df.users[:3], fracs = [0.1,0.2,0.1])
# df.data = create_session(df.data, config)
# df.data = clean_data(df.data, config)
# df.data = cut_sessions(df.data, config)
# config.clean_mode = ['rm_small_sub_session']
# df.data = clean_data(df.data, config)
# df.data

# Model Prepration

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from dataloader import Music_data, create_data_loader
import torch.optim as optim
import torch.nn.functional as F
import Model
import random
from Model import create_embeding, Seq2Seq, Encoder, Decoder

In [9]:
seed = 0
"""Set seed"""
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
os.environ["PYTHONHASHSEED"] = str(seed)

In [50]:
config.num_epochs = 1000
config.batch_size = 50
config.learning_rate = 0.001

In [44]:
embedding, dim, _ = Model.create_embeding(config)
train_iterator = create_data_loader(config, 'train', shuffle = True)
test_iterator = create_data_loader(config, 'test', shuffle = True)
valid_iterator = create_data_loader(config, 'valid', shuffle = True)
itr = iter(train_iterator)
# df = Music_data(config, 'train')

In [45]:
enc = Encoder(config).to(config.device)
dec = Decoder(config).to(config.device)
model = Seq2Seq(config, enc, dec).to(config.device)

In [46]:
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
pad_idx = config.pad_index
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [55]:
# Tensorboard to get nice loss plot
writer = SummaryWriter(f"runs/loss_plot")
step = 0

In [52]:
# Overfit a single batch
batch = next(iter(train_iterator))

In [56]:
tic = time.perf_counter()
for epoch in range(config.num_epochs):
#     for batch_idx, batch in enumerate(train_iterator):
    _, source, target = batch
    source = (source[0].to(config.device), source[1].to(config.device))
    target = (target[0].to(config.device), target[1].to(config.device))
    output = model(config, source, target)

    output = output[1:].reshape(-1, output.shape[2])
    target = target[0][1:].reshape(-1)

    optimizer.zero_grad()
    loss = criterion(output, target)

    # Back prop
    loss.backward()
    if epoch%20 == 0:
        print(loss.item())

    # Clip to avoid exploding gradient issues, makes sure grads are
    # within a healthy range
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Gradient descent step
    optimizer.step()

    # Plot to tensorboard
    writer.add_scalar("Training loss", loss, global_step=step)
    step += 1
toc = time.perf_counter()
print(f'Performance: {toc - tic:0.8f} seconds')

0.012327116914093494
0.012321611866354942
0.012316377833485603
0.012315361760556698
0.012310433201491833
0.01230187900364399
0.012297390960156918
0.012293086387217045
0.012288909405469894
0.012284868396818638
0.012280971743166447
0.012277207337319851
0.01227356493473053
0.012270040810108185
0.012266633100807667
0.01226332038640976
0.012263131327927113
0.012257017195224762
0.01225692592561245
0.01225114706903696
0.012251116335391998
0.012245637364685535
0.0122429970651865
0.012240436859428883
0.012240491807460785
0.012235532514750957
0.012233204208314419
0.012230923399329185
0.012231077067553997
0.012226566672325134
0.012224476784467697
0.012222427874803543
0.012221090495586395
0.012221379205584526
0.012219298630952835
0.01221501361578703
0.012213167734444141
0.012211400084197521
0.012209679000079632
0.012209980748593807
0.012206382118165493
0.012204792350530624
0.012205123901367188
0.01220356673002243
0.012200291268527508
0.012198833748698235
0.012197425588965416
0.012197763659060001
0